# Домашняя работа №3

В данном ноутбуке мы сосредоточимся на таком аспекте как Text Mining


### Программа

Цель: дать характеристику описанным текстам на небольшой выборке релевантных видео

1. [Подготовительный этап](#preparing)

    1.1 Загрузка данных

    1.2 Предобработать тексты с помощью

2. [Этап "Тематическое моделирование"](#themes)

    2.1 Векторизовать тексты для получения тематического моделирования

    2.1 Провести базовое тематическое моделирование

3. [Этап "Анализ сентиментов"](#sentiment)

    3.1 Провести классификацию сентиментов с помощью предобученной модели

    3.2 Проанализировать данные применительно к данным

4. [Заключительный этап](#end)


### Описание данных:

- `video_data.xlsx` - одни из самых популярных отобранных видео по тематике
- `comments.xlsx` - собранный массив сырых комментариев к этим видео


In [1]:
# 0 определение вспомогательных функций

# download stopwords corpus, you need to run it once
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import string
import re


#Create lemmatizer and stopwords list
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

", ".join(russian_stopwords)



PUNCT_TO_REMOVE = string.punctuation

def remove_tabs(text): #убираем дичь со специальными символами
    return text.translate(str.maketrans("\n\t\r", "   "))

def remove_punct(text):
    return re.sub(r'[^a-zA-Zа-яА-ЯёЁ]+',' ', text)

def remove_urls(text):
    return re.sub(r'http\S+', '', text, flags=re.MULTILINE)

def remove_mentions(text):
    return re.sub("@[A-Za-z0-9_]+","", text)

def remove_hashtags(text):
    return re.sub("#[A-Za-z0-9_]+","", text)

def clean_text(text: str)->str:
    text = remove_urls(text)
    text = remove_tabs(text)
    text = remove_mentions(text)
    text = remove_hashtags(text)

    return text


def preprocess_text(text: str) -> str:

    text = remove_punct(text)

    tokens = mystem.lemmatize(text.lower())

    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## <a id="preparing">Этап 1. Подготовка данных</a>

### Загрузка данных

In [2]:
# 1 загрузка данных

import pandas as pd

video_df = pd.read_excel('video_data.xlsx')

video_df.head()

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,contentDetails.licensedContent,contentDetails.projection,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,player.embedHtml,topicDetails.topicCategories,snippet.defaultLanguage,origin
0,youtube#video,39bfg2o1oxyiRqGPtZrCWyuwH7o,vzdPoXX2UBM,2023-05-23T12:19:23Z,UCXNXpk6BPZO-M-fYyvfF06Q,"ЧТО НОВОГО K-BEAUTY? РЕТИНОЛ, БАКУЧИОЛ, ЭКОЛОГ...","Дорогие, привет!\nКак много всего происходит в...",https://i.ytimg.com/vi/vzdPoXX2UBM/default.jpg,120,90,...,True,rectangular,10728,1068,0,62,"<iframe width=""480"" height=""270"" src=""//www.yo...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,Европа
1,youtube#video,--pazyNlbv_hVjU7gHSLmO6X4NM,zAUlL_UD2wI,2023-03-12T06:00:28Z,UCtHLW7MItjNZXYsmGA1mg9g,Как всегда выглядеть хорошо без макияжа? 12 пр...,ССЫЛКИ:\n✨ Масло-термозашита для волос (промок...,https://i.ytimg.com/vi/zAUlL_UD2wI/default.jpg,120,90,...,True,rectangular,345863,21823,0,544,"<iframe width=""480"" height=""270"" src=""//www.yo...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,ru,Европа
2,youtube#video,9NzoyyTqtOiPLU-vwWlBxWVXeys,MQhiHSFflM8,2023-05-25T16:00:55Z,UCDiixmeeY6BQvVVJfkNxcMQ,ЧЕСТНО О БЬЮТИ БОМБ x Дарья Граф 🍄 Новая колле...,#бьютибомб #дашаграф #beautybomb #косметика\n\...,https://i.ytimg.com/vi/MQhiHSFflM8/default.jpg,120,90,...,True,rectangular,45067,3762,0,298,"<iframe width=""480"" height=""270"" src=""//www.yo...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,Европа
3,youtube#video,6YaZ-26bJMjogDeqhRnFnUsmV-w,DBQikr47VDY,2023-06-01T18:38:50Z,UCU0sNUPabsC4yQm0sHTVOcg,✨Милый макияж с розовыми тенями | Cute makeup ...,"Зайка привет, моя хорошая💖🥹 Спасибо что зашла ...",https://i.ytimg.com/vi/DBQikr47VDY/default.jpg,120,90,...,False,rectangular,232,24,0,2,"<iframe width=""480"" height=""270"" src=""//www.yo...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,Европа
4,youtube#video,K5YpiZT3XnNs5vFJ1WftANgcpzs,4kbxKnzhstc,2023-05-16T08:30:03Z,UCKoFMFiKx297VCuHB29mMXw,КАК Я стала ЛИЦОМ BEAUTY BOMB?😳,Beauty Bomb дарят новую коллекцию Acid Summer ...,https://i.ytimg.com/vi/4kbxKnzhstc/default.jpg,120,90,...,True,rectangular,747714,45204,0,1116,"<iframe width=""480"" height=""270"" src=""//www.yo...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,Европа


In [3]:
# 2 загрузка данных комментариев

cmt_df = pd.read_excel('comments.xlsx')

cmt_df = cmt_df.dropna(subset=['snippet.textOriginal'])

cmt_df

,kind,etag,id,snippet.videoId,snippet.textDisplay,snippet.textOriginal,snippet.authorDisplayName,snippet.authorProfileImageUrl,snippet.authorChannelUrl,snippet.authorChannelId.value,...,snippet.likeCount,snippet.publishedAt,snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic,replies.comments,snippet.moderationStatus,real_comments_cnt,snippet.parentId
0,youtube#commentThread,lqN0Zzupy-0f_AAASt7gKLOZalc,Ugz57x-YZSYddlWifIt4AaABAg,vzdPoXX2UBM,До Хян! Спасибо большое за новое видео 🫶🏻 обож...,До Хян! Спасибо большое за новое видео 🫶🏻 обож...,Alex Yost,https://yt3.ggpht.com/ytc/AGIKgqNjg7gvFa7b6j1c...,http://www.youtube.com/channel/UCTaEjHhPIMJXRN...,UCTaEjHhPIMJXRNzqcx54jtg,...,0,2023-06-01T21:53:57Z,2023-06-01T21:53:57Z,1.0,0.0,1.0,NaN,NaN,0.0,-1
1,youtube#commentThread,FSr6OFvRbevOro7VVmOb-ANmVF0,Ugzqfh0Visn0OYf_EZN4AaABAg,vzdPoXX2UBM,Бо Хян онни❤на озоне продавали товары от вас-B...,Бо Хян онни❤на озоне продавали товары от вас-B...,Aska Yuska,https://yt3.ggpht.com/ytc/AGIKgqNbP5fGCsLeQA3b...,http://www.youtube.com/channel/UCIKYk4-5rFZs3r...,UCIKYk4-5rFZs3ry8zgJesHA,...,1,2023-05-31T08:03:03Z,2023-05-31T08:09:51Z,1.0,0.0,1.0,NaN,NaN,0.0,-1
2,youtube#commentThread,ycRzG8zBi19WCF6USiBGOR_9l5w,UgxAyZkv4V1m1sm-aUt4AaABAg,vzdPoXX2UBM,Классные средства! В вашем магазине на Озон бу...,Классные средства! В вашем магазине на Озон бу...,Ирина Крылова,https://yt3.ggpht.com/ytc/AGIKgqP2ASGZlu-D33_F...,http://www.youtube.com/channel/UC1yEbLPPcFEZX6...,UC1yEbLPPcFEZX6RRuWwjp4g,...,0,2023-05-30T18:58:19Z,2023-05-30T18:58:19Z,1.0,0.0,1.0,NaN,NaN,0.0,-1
3,youtube#commentThread,taeWHMsCs91UCrTujiIuJEGrmcs,UgwHEEpAHbCqWdjzy494AaABAg,vzdPoXX2UBM,спасибо за видео),спасибо за видео),Ustalaya,https://yt3.ggpht.com/ytc/AGIKgqNTnXNpcPMQpsvH...,http://www.youtube.com/channel/UCJU-KEQgK7Y14P...,UCJU-KEQgK7Y14POdao_6uCg,...,0,2023-05-28T18:54:48Z,2023-05-28T18:54:48Z,1.0,0.0,1.0,NaN,NaN,0.0,-1
4,youtube#commentThread,6XEm3qL2GdKiNxA2Lkmnrnb4P3I,Ugz2C__gg0nt3MuuZXZ4AaABAg,vzdPoXX2UBM,А можно будет сделать обзор про средства для п...,А можно будет сделать обзор про средства для п...,Olga M,https://yt3.ggpht.com/ytc/AGIKgqPJ5KmbaRTSJwLo...,http://www.youtube.com/channel/UCURgXlOeRejxcY...,UCURgXlOeRejxcYKwZU-vz7w,...,3,2023-05-28T10:35:17Z,2023-05-28T10:35:17Z,1.0,0.0,1.0,NaN,NaN,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8492,youtube#comment,SrLNUEzqeGp81VWedKPEbMDJBq4,Ugw_CW1RnR68GyuxVEl4AaABAg.9kGNFIgGa7Z9mhGVoE-2yy,BFcjeM3V3dc,@🦋Полина 🦋 💖,@🦋Полина 🦋 💖,Марьям,https://yt3.ggpht.com/gFdhsGyMU2GXNAZLOYO4gtqJ...,http://www.youtube.com/channel/UCQUuAGFrJthPST...,UCQUuAGFrJthPSTu8GeF67oQ,...,0,2023-03-01T03:07:00Z,2023-03-01T03:07:00Z,NaN,NaN,NaN,NaN,NaN,NaN,Ugw_CW1RnR68GyuxVEl4AaABAg
8493,youtube#comment,jQEOTB8DJ1NQkjRXIajt_wklkDQ,Ugw_CW1RnR68GyuxVEl4AaABAg.9kGNFIgGa7Z9mhGUf7zXrY,BFcjeM3V3dc,@Margo Aravin 💓,@Margo Aravin 💓,Марьям,https://yt3.ggpht.com/gFdhsGyMU2GXNAZLOYO4gtqJ...,http://www.youtube.com/channel/UCQUuAGFrJthPST...,UCQUuAGFrJthPSTu8GeF67oQ,...,0,2023-03-01T03:06:51Z,2023-03-01T03:06:51Z,NaN,NaN,NaN,NaN,NaN,NaN,Ugw_CW1RnR68GyuxVEl4AaABAg
8494,youtube#comment,80_51kGA0llynKoz12AVpUyYvjk,Ugw_CW1RnR68GyuxVEl4AaABAg.9kGNFIgGa7Z9mdL-sHQSpe,BFcjeM3V3dc,И я,И я,Margo Aravin,https://yt3.ggpht.com/ytc/AGIKgqOBhMBgYvBwoSyA...,http://www.youtube.com/channel/UC1ar9sOW7t0niW...,UC1ar9sOW7t0niWfkWHeqcBQ,...,2,2023-02-27T14:29:22Z,2023-02-27T14:29:22Z,NaN,NaN,NaN,NaN,NaN,NaN,Ugw_CW1RnR68GyuxVEl4AaABAg
8495,youtube#comment,sdw_wTTarV0WBX4404R5jUy912Y,Ugw_CW1RnR68GyuxVEl4AaABAg.9kGNFIgGa7Z9mbG4-_uJfw,BFcjeM3V3dc,Это Я,Это Я,🦋Полина 🦋,https://yt3.ggpht.com/432OqfqNjILbDGMPCCcTTnPq...,http://www.youtube.com/channel/UCJoCrWbCSkZMgZ...,UCJoCrWbCSkZMgZadzCLfzfQ,...,1,2023-02-26T19:07:46Z,2023-02-26T19:07:46Z,NaN,NaN,NaN,NaN,NaN,NaN,Ugw_CW1RnR68GyuxVEl4AaABAg


### Text Preprocessing

In [4]:
# 3 Формирование корпуса текстов

corpora = list(cmt_df['snippet.textOriginal'].values)


In [5]:
# 4 Preprocessing with simple text clearance for

cleaned_text = [clean_text(t) for t in corpora]

cmt_df['cleaned_text'] = cleaned_text

In [6]:
# 5 Preprocessing with lemmatization

lemmatized = [preprocess_text(s) for s in cleaned_text]

cmt_df['lemmatized_text'] = lemmatized

In [7]:
## 6 Оценка количества слов в лемматизации

cmt_df['lem_word_count'] = cmt_df['lemmatized_text'].apply(lambda x: len(x.split()))
cmt_df['lem_sym_count'] = cmt_df['lemmatized_text'].apply(lambda x: len(x))

cmt_df['lem_word_count'].describe()

count    8495.000000
mean       10.296410
std        13.438203
min         0.000000
25%         3.000000
50%         6.000000
75%        12.000000
max       295.000000
Name: lem_word_count, dtype: float64

***Предлагаем оставить больше 3 слов на каждый для более точного смыслового определения***

In [8]:
# 7 Фильтр
cmt_df = cmt_df[cmt_df['lem_word_count'] >= 3]


## <a id="sentiment">Этап 2. Сентимент анализ</a>

### Содержательное предположение

***Корейская косметика в обзорах воспринимается более позитивно, чем европейская***

### Дизайн эксперимента

Для нашей гипотезы о различии реакций на корейскую и европейскую косметику нами были выбраны отдельные видео и к ним загружены комментарии

Давайте сравним реакцию на видео относительно их контента

**Группы:**

1. Видео о корейской косметике
2. Видео о европейской косметике
3. Видео о том и другом (мульти-обзоры)

Для проведения sentiment анализа мы будем использовать предобученную модель из библиотеки `transformers`

Итогом станет сравнение реакций в двух группах и верификация относительно контрольной




In [9]:
# 10 Статистика

video_df['origin'] = video_df['origin'].map({'Европа': 'Европа', 'Корея': 'Корея', 'И то и т о': 'Контроль'})



video_df['origin'].value_counts(ascending=False)


origin
Европа      12
Корея       12
Контроль     9
Name: count, dtype: int64

In [10]:
# 11 Сравнение групп по статистике
import numpy as np


video_df.pivot_table(index=['origin'], values=['statistics.likeCount', 'statistics.viewCount', 'statistics.commentCount'], aggfunc=[np.mean, np.sum])

mean                                             
         statistics.commentCount statistics.likeCount statistics.viewCount   
origin                                                                       
Европа                227.666667          7456.000000        134729.583333  \
Контроль              293.555556          6034.888889        156458.777778   
Корея                 263.250000          7523.416667        104419.833333   

                             sum                                            
         statistics.commentCount statistics.likeCount statistics.viewCount  
origin                                                                      
Европа                      2732                89472              1616755  
Контроль                    2642                54314              1408129  
Корея                       3159                90281              1253038

**В группах сопоставимое число просмотров, лайков и комментариях как по средним, так и по сумме**

***Примечание: в рамках учебного скрипта такое утверждение принимается за верное, однако для верифицируемости может использоваться как большее число видео, так и статистические критерии на проверку неравенства средних в группах по критерию ${\chi}^2$***


### Sentiment Transformers

В качестве предобученной модели мы используем Bert Cased от cointegrated (Facebook AI Research)

[source]https://huggingface.co/cointegrated/rubert-tiny-sentiment-balanced)

Причины выбора модели:
1) Она обучалась на значительно большем количестве текстов, чем можно сформировать в рамках учебной работы
2) Она обучалась, в том числе на продуктовых отзывах
3)

In [11]:
!pip install torch
!pip install transformers
!pip install tqdm


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [12]:
# 12 Инициализация моделей и её зависимостей
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def get_sentiment(text, return_type='label'):
    """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba



In [13]:
# Классификация по сентименту

from tqdm import tqdm

sent_scores = []
sent_labels = []


for text in tqdm(cmt_df['cleaned_text']):

    # Adding score
    label = get_sentiment(text, 'label')
    # Adding score
    score = get_sentiment(text, 'score')

    sent_scores.append(score)
    sent_labels.append(label)


100%|██████████| 6836/6836 [00:53<00:00, 128.96it/s]


In [23]:
# 13 Маппинг всего и сразу

mapper_df =  video_df[['id', 'origin']].copy().drop_duplicates()

mapper_df = mapper_df.rename(columns={'id':'snippet.videoId'})

cmt_df['label'] = sent_labels
cmt_df['score'] = sent_scores

cmt_df = cmt_df.merge(mapper_df, how='left', left_on='snippet.videoId', right_on='snippet.videoId')

cmt_df

/var/folders/s5/4r_7wwf925g2bnv4060z0x400000gn/T/ipykernel_15605/898512325.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmt_df['label'] = sent_labels
/var/folders/s5/4r_7wwf925g2bnv4060z0x400000gn/T/ipykernel_15605/898512325.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmt_df['score'] = sent_scores


,kind,etag,id,snippet.videoId,snippet.textDisplay,snippet.textOriginal,snippet.authorDisplayName,snippet.authorProfileImageUrl,snippet.authorChannelUrl,snippet.authorChannelId.value,...,snippet.moderationStatus,real_comments_cnt,snippet.parentId,cleaned_text,lemmatized_text,lem_word_count,lem_sym_count,label,score,origin
0,youtube#commentThread,lqN0Zzupy-0f_AAASt7gKLOZalc,Ugz57x-YZSYddlWifIt4AaABAg,vzdPoXX2UBM,До Хян! Спасибо большое за новое видео 🫶🏻 обож...,До Хян! Спасибо большое за новое видео 🫶🏻 обож...,Alex Yost,https://yt3.ggpht.com/ytc/AGIKgqNjg7gvFa7b6j1c...,http://www.youtube.com/channel/UCTaEjHhPIMJXRN...,UCTaEjHhPIMJXRNzqcx54jtg,...,NaN,0.0,-1,До Хян! Спасибо большое за новое видео 🫶🏻 обож...,хян спасибо большой новый видео обожать просто...,50,385,negative,-0.102212,Европа
1,youtube#commentThread,FSr6OFvRbevOro7VVmOb-ANmVF0,Ugzqfh0Visn0OYf_EZN4AaABAg,vzdPoXX2UBM,Бо Хян онни❤на озоне продавали товары от вас-B...,Бо Хян онни❤на озоне продавали товары от вас-B...,Aska Yuska,https://yt3.ggpht.com/ytc/AGIKgqNbP5fGCsLeQA3b...,http://www.youtube.com/channel/UCIKYk4-5rFZs3r...,UCIKYk4-5rFZs3ry8zgJesHA,...,NaN,0.0,-1,Бо Хян онни❤на озоне продавали товары от вас-B...,бо хян онни озон продавать товар bonya s pouch...,26,184,negative,-0.963522,Европа
2,youtube#commentThread,ycRzG8zBi19WCF6USiBGOR_9l5w,UgxAyZkv4V1m1sm-aUt4AaABAg,vzdPoXX2UBM,Классные средства! В вашем магазине на Озон бу...,Классные средства! В вашем магазине на Озон бу...,Ирина Крылова,https://yt3.ggpht.com/ytc/AGIKgqP2ASGZlu-D33_F...,http://www.youtube.com/channel/UC1yEbLPPcFEZX6...,UC1yEbLPPcFEZX6RRuWwjp4g,...,NaN,0.0,-1,Классные средства! В вашем магазине на Озон бу...,классный средство ваш магазин озон новинка mix...,7,50,positive,0.862316,Европа
3,youtube#commentThread,6XEm3qL2GdKiNxA2Lkmnrnb4P3I,Ugz2C__gg0nt3MuuZXZ4AaABAg,vzdPoXX2UBM,А можно будет сделать обзор про средства для п...,А можно будет сделать обзор про средства для п...,Olga M,https://yt3.ggpht.com/ytc/AGIKgqPJ5KmbaRTSJwLo...,http://www.youtube.com/channel/UCURgXlOeRejxcY...,UCURgXlOeRejxcYKwZU-vz7w,...,NaN,0.0,-1,А можно будет сделать обзор про средства для п...,сделать обзор средство проблема мешок глаз тем...,15,110,neutral,-0.250969,Европа
4,youtube#commentThread,JR1nxqvPS4CNLG9TN9ndWcN3t_A,Ugx7wn9TGfLnjUweill4AaABAg,vzdPoXX2UBM,"Спасибо! Очень полезная информация, особенно п...","Спасибо! Очень полезная информация, особенно п...",Olga M,https://yt3.ggpht.com/ytc/AGIKgqPJ5KmbaRTSJwLo...,http://www.youtube.com/channel/UCURgXlOeRejxcY...,UCURgXlOeRejxcYKwZU-vz7w,...,NaN,0.0,-1,"Спасибо! Очень полезная информация, особенно п...",спасибо очень полезный информация особенно кре...,9,66,neutral,0.357816,Европа
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6831,youtube#comment,xS3P57mbgth2gH5i880K8yahTJ8,UgyyAVRq5tzV5Sft8Eh4AaABAg.9qK84NHl_3M9qKAQ2uusLy,KDhOg88LRxE,Мне вообще показалось что Миша далеко не всем ...,Мне вообще показалось что Миша далеко не всем ...,mocco locco,https://yt3.ggpht.com/ytc/AGIKgqNvwyagHDeSqb7_...,http://www.youtube.com/channel/UCADLBK7x0Ecelq...,UCADLBK7x0EcelqRWng9ELZg,...,NaN,NaN,UgyyAVRq5tzV5Sft8Eh4AaABAg,Мне вообще показалось что Миша далеко не всем ...,вообще показываться миша далеко подходить,5,41,negative,-0.875127,Контроль
6832,youtube#comment,9m59SN263tMeHLpIRq2oYneXWLE,UgyyAVRq5tzV5Sft8Eh4AaABAg.9qK84NHl_3M9qKA-AK7mFh,KDhOg88LRxE,"Мне показалось, что жирновато выглядит на этом...","Мне показалось, что жирновато выглядит на этом...",Stacy,https://yt3.ggpht.com/ytc/AGIKgqNYYf3wYLXdLxe2...,http://www.youtube.com/channel/UCqKn84x_Bc51na...,UCqKn84x_Bc51naCD5v95aGw,...,NaN,NaN,UgyyAVRq5tzV5Sft8Eh4AaABAg,"Мне показалось, что жирновато выглядит на этом...",показываться жирноватый выглядеть это видео,5,43,neutral,-0.083332,Контроль
6833,youtube#comment,Y1uTlzueYFXyzYF3EwvGzZXy6lM,UgyyAVRq5tzV5Sft8Eh4AaABAg.9qK84NHl_3M9qK9mo_XP32,KDhOg88LRxE,Тоже вот подобное помню из старых видео...у ме...,Тоже вот подобное помню из старых ви

In [24]:
# 14 Cравнение средних в пивоте

cmt_df.pivot_table(index=['origin'], values=['score'], aggfunc=['count', np.mean, np.median, np.var]).T

,origin,Европа,Контроль,Корея
count,score,2054.000000,2142.000000,2640.000000
mean,score,0.077952,0.053981,0.005008
median,score,0.064647,0.033975,-0.046326
var,score,0.345066,0.354517,0.352750


### Гипотезы

Разновидность

H0: $μ1 = μ2$ (population mean of Group A is equal to Group B)
H1: $μ1 \neq μ2$ (population mean of Group A is different from Group B)

In [25]:
# 15 Cравнение средних и T-test

import scipy.stats as stats

group_1 = cmt_df[(cmt_df['origin'] == 'Корея') & (cmt_df['snippet.parentId'] == -1) ]['score']
group_2 = cmt_df[(cmt_df['origin'] == 'Европа') & (cmt_df['snippet.parentId'] == -1) ]['score']

# Perform the two sample t-test with equal variances
test_res = stats.ttest_ind(a=group_1, b=group_2, equal_var=False)

print('Total Samples:', len(group_1), 'Group 1 mean:', group_1.mean())
print('Total Samples:', len(group_2), 'Group 2 mean:', group_2.mean())
print('Statistics', test_res.statistic)
print('P-Value', test_res.pvalue < .01)

Total Samples: 1750 Group 1 mean: 0.03735124403291515
Total Samples: 1537 Group 2 mean: 0.11283258239756215
Statistics -3.6497196017645646
P-Value True


### Интерпретация

**T-test на равенство средних, показа что средний сентимент скор для Европы выше, чем для Кореи на статистически значимом уровне.**

Это могло произойти по следующим причинам:

- Выборка видео не репрезентативна, поскольку взято достаточно малое количество Видео для группы A и для группы B -> малое число степеней свободы для видео
- Блогер может переносить свою личность на восприятие косметики
- Модель отрабатывает хуже в данном кейсе и нуждается в дополнительной подстройке

**Содержательные выводы и направления дальнейших исследований**

1. ***Содержательная гипотеза опровергнута***
2. Видео с европейской в имеющейся выборке воспринимается более позитивно, чем с корейской судя по родительским комментариям
3. Для исключения фактора влияния блогера следует взять разные видео от одного и того же блогера, чтобы нивелировать влияние фактора его личности
4. Стоит попробовать альтернативные модели машинного обучения и выявить средние также по ним
5. Стоит попробовать протестировать на отзывах на маркетплейсах, тогда будет только фактор площадки (цены, впечатления от сервиса и доставки)

## <a id="topics">Этап 3. Тематическое моделирование</a>

Для задачи тематического моделирования мы будем применять метод LDA (Latent Dirichlet Analysis) из модуля `gensim`

In [26]:
!pip install gensim
!pip install pyLDAvis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [27]:
# 16 Токенизация и формирование корпуса
from nltk import word_tokenize

all_text = [word_tokenize(line) for line in cmt_df.lemmatized_text.values]


In [28]:
# 17 Подготовка данных для модели LDA

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora
from pprint import pprint


# Создание словаря - фактически маппинга токенов
id2word = corpora.Dictionary(all_text)

# Создание токенов
texts = all_text

# Матрица документы-частоты
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)]]


In [29]:
# 18 Обучение моделей

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=42,
                                           chunksize=100,
                                           alpha='auto',
                                            eta = 'auto',
                                           per_word_topics=True)

# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.033*"кожа" + 0.011*"это" + 0.011*"средство" + 0.009*"ничто" + '
  '0.009*"заказывать" + 0.008*"день" + 0.008*"бренд" + 0.007*"разный" + '
  '0.007*"пользоваться" + 0.006*"брать"'),
 (1,
  '0.023*"косметика" + 0.018*"очень" + 0.016*"видео" + 0.016*"весь" + '
  '0.013*"это" + 0.012*"мочь" + 0.012*"хороший" + 0.010*"продукт" + '
  '0.009*"спасибо" + 0.008*"ваш"'),
 (2,
  '0.032*"это" + 0.015*"человек" + 0.012*"просто" + 0.011*"свой" + '
  '0.010*"который" + 0.010*"понимать" + 0.009*"подходить" + 0.008*"консилер" + '
  '0.008*"знать" + 0.007*"комментарий"')]


In [30]:
# 19 Вычисление метрик модели

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=all_text, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.28472003184794
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | fal

In [31]:
# 20 Визуализация топиков

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.169012  0.004571       1        1  42.598378
2      0.080079 -0.133157       2        1  29.895065
0      0.088933  0.128586       3        1  27.506557, topic_info=          Term        Freq       Total Category  logprob  loglift
154       кожа  833.000000  833.000000  Default  30.0000  30.0000
84   косметика  849.000000  849.000000  Default  29.0000  29.0000
87       очень  682.000000  682.000000  Default  28.0000  28.0000
505    человек  384.000000  384.000000  Default  27.0000  27.0000
8        видео  617.000000  617.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
73        лицо  114.212855  304.630087   Topic3  -5.3247   0.3097
238     каждый   84.659953  164.220835   Topic3  -5.6241   0.6282
474   говорить   89.887923  293.005447   Topic3  -5.5642   0.1091
22        мочь  101.631141  663.539995   Topic3  -5.4414  -0.5855
112       весь  102.100542  726.781761   Topic3  -5.4368  -0.6719

[196 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
7         1  0.952816      большой
7         2  0.042036      большой
1507      1  0.992296        бомба
1507      2  0.011149        бомба
507       2  0.020748        брать
...     ...       ...          ...
576       2  0.977894  чувствовать
65        1  0.301574          это
65        2  0.534254          это
65        3  0.164436          это
194       3  0.987218       эффект

[335 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

#### По итогам получилось 3 топика с базовыми параметрами - также как в базовом случае

### Интерпретация LDA

Perplexity - -8.28472003184794
Coherence Score - 0.52



Topic 1 - Отзывы на обзоры блогера
Topic 2 - Отзывы о средстве на кожу
Topic 3 - Отзывы о средстве в общем


In [20]:
# 21 predicting new text which is in text dataframe


results = []

for t in all_text:
    corp = id2word.doc2bow(t)
    pred = [x[1] for x in lda_model[corp][0]]
    results.append(pred)

In [21]:
# 22 Adding topics to text indexes and original text

topics = pd.DataFrame(data=results, columns=['Topic_1', 'Topic_2', 'Topic_3'])

topics['index'] = cmt_df.index
topics['snippet.textOriginal'] = cmt_df['snippet.textOriginal']
topics['lemmatized_text'] = cmt_df['lemmatized_text']

In [22]:
# 23 Saving

topics.to_excel('comments_with_topic.xlsx', index=False)

## <a id="sentiment">Заключение</a>

- Нами был проведен анализ текстовых комментанриев к разного рода видео `video_data.xlsx` с указанным объектом обзора

- Для обработки текста использовались различные подходы очистки, включая удаление пунктуации, url. Также для улучшения качества модели на текстовых данных мы использовали лемматизацию

- Был получен массив данных `cleared_comments.xlsx`

- Был получен массив данных Документ-частота токена `cm_cmatrix.xlsx`

- С помощью сентимент анализа было получено, что европейская косметика воспринимается более позитивно, чем корейская. Это противоречит нашей содержательной гипотезе. Мы перечисли несколько факторов почему наша содержательная гипотеза не подтвердилась:
    - Влияние Youtube как площадки
    - Влияние блогера - лучше сравнивать у одного
    - Проблемы смещения модели
    - Малое количество видео

- Была обучена и визуализирована модель LDA с показателями метрик Perplexity - -8.28472003184794, Coherence Score - 0.52

- В рамках тематического моделирования были получены 3 топика:
    Topic 1 - Отзывы на обзоры блогера
    Topic 2 - Отзывы о средстве на кожу
    Topic 3 - Отзывы о средстве в общем


### Бонус:Тюнинг модели LDA


### Справка по тюнингу

`Coherence Score` (Показатель согласованности) - это показатель того, насколько хорошо темы в модели скрытого распределения Дирихле (LDA) связаны друг с другом. Он рассчитывается путем измерения сходства между словами в каждой теме. Более высокий балл согласованности указывает на то, что темы в большей степени связаны друг с другом, а более низкий балл согласованности указывает на то, что темы в меньшей степени связаны друг с другом.

Существует ряд различных показателей согласованности, которые можно использовать, но наиболее распространенными из них являются:

- **C-value**

- **UMass**

- **Sparsity (Разреженность)**

***Важно отметить, что показатель согласованности - это всего лишь один из показателей качества модели. Другие показатели, такие как `Perplexity` (недоумение), также могут быть использованы для оценки качества модели.***